In [100]:
import xml.etree.ElementTree as ET
from datetime import datetime
from googlesearch import search
import time
import requests
import json
import re
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [101]:
def remove_prefix_from_name(name):

    name = name + ' '
    
    suffixes = [' BBA ', ' LLB ', " MSc ", " MA ", " BA ", " RA "]
    for suffix in suffixes:
        if suffix in name:
            name = name.replace(suffix[:-1], suffix[:-1]+'.')

    
    
    pattern = r"(?!ij|th|ph|hr|ch|jr|sr)[a-z]{2,}\.\s?"
    prefixes = re.findall(pattern, name, flags=re.IGNORECASE)
    
    if prefixes:
        name_without_prefixes = re.sub(pattern, '', name, flags=re.IGNORECASE).strip()
        
        
        return prefixes, name_without_prefixes
    else:
        return '', name.strip()
    
def xpath(element, path, namespaces):
    obj = element.find(path, namespaces)
    
    if obj is not None:
        return obj.text
    else:
        return ''
    
def xpath_adres(element, namespaces, Type):
    
    Incomplete = False
    
    if Type == 'Bezoek':
    
        obj = element.find('p:adressen/p:adres[p:type="Bezoekadres"]', namespaces)

        if obj is not None:
            
            straat = obj.find('p:straat', namespaces)
            if straat is not None:
                straat = straat.text
            else:
                Incomplete = True
            
            huisnummer = obj.find('p:huisnummer', namespaces)
            if huisnummer is not None:
                huisnummer = huisnummer.text
            else:
                Incomplete = True
            
            postcode = obj.find('p:postcode', namespaces)
            if postcode is not None:
                postcode = postcode.text
            else:
                Incomplete = True
                
            plaats = obj.find('p:plaats', namespaces)
            if plaats is not None:
                plaats = plaats.text.title()
                
            else:
                Incomplete = True
                
            if Incomplete == True:
                return ''

            return f"{straat} {huisnummer} {postcode} {plaats}"
         
        else:
            return ''
        
    if Type == 'Post':
        
        obj = element.find('p:adressen/p:adres[p:type="Postadres"]', namespaces)
        
        if obj is not None:
            postbus = obj.find('p:postbus', namespaces)
            if postbus is not None:
                postbus = postbus.text
            else:
                Incomplete = True
            
            postcode = obj.find('p:postcode', namespaces)
            if postcode is not None:
                postcode = postcode.text
            else:
                Incomplete = True
            
            plaats = obj.find('p:plaats', namespaces)
            if plaats is not None:
                plaats = plaats.text.title()
                
            else:
                Incomplete = True
                
            if Incomplete == True:
                return ''
            

            return f"{postbus} {postcode} {plaats}"
         
        else:
            return ''
        
    else:
        return ''
        
def xpath_TOOI(element, path, namespaces, afkorting):
    
    dc_publisher = element.find(path, namespaces)
    if dc_publisher is not None:

        # hier wordt de afkorting gebruikt bijv. ws = waterschappen
        if dc_publisher.text.rsplit('/')[-1][:len(afkorting)] == afkorting:
            
            return dc_publisher.text.rsplit('/')[-1]
        else:
            return ''
    else:
        return ''
    
    
def xpath_naam(element, path, namespaces):
    # verzamel de naam in de XML
    initial_name = element.find(path, namespaces)
    
    if initial_name is not None:
        initial_name_txt = initial_name.text
        
        pattern = r'\((.*?)\)'
        matches = re.findall(pattern, initial_name_txt)
        
        initial_name_txt = re.sub(pattern, '', initial_name_txt)
        if " (" in initial_name_txt:
            initial_name_txt = initial_name_txt.split(' (')[0]
        
        if '  ' in initial_name_txt:
            initial_name_txt = initial_name_txt.replace('  ', ' ')

        # er staat vaak een prefix als dr. of mew. voor deze wordt verwijderd?
        prefixes, name_without_prefix = remove_prefix_from_name(initial_name_txt)
        name_without_prefix = name_without_prefix.lstrip()

        initial_name_split = name_without_prefix.split(' ', 1)

        # bij gemeenten staat er een persoon in als mew. Schouten dus zonder initialen. 
        if len(initial_name_split) == 2:
            first_word = initial_name_split[0]
            if '.' not in first_word and len(first_word) != 1:

                foaf_firstName = initial_name_split[0]
                foaf_initials = foaf_firstName[0].upper() + '.'
            else:
                foaf_initials = initial_name_split[0]
                foaf_firstName = ''

            foaf_lastName = initial_name_split[1]
            x = foaf_lastName.split(' ')
            if x[0] != '':
                if x[0][-1] == '.':
                    foaf_lastName = ''.join(x[1:])
                    foaf_initials = foaf_initials + x[0]
        
        else:
            foaf_initials = ''
            foaf_firstName = ''
            foaf_lastName = initial_name_split[0]
    
    # gebeurt in de praktijk niet maar maakt het wel failproof
    else:
        foaf_initials = ''
        foaf_firstName = ''
        foaf_lastName = ''
        
    prefixes = ''.join(prefixes)
    prefixes = prefixes.rstrip()
    
    if matches != []:
        if len(matches[0].split(' ')) == 1:
            foaf_firstName = matches[0]
            
            if '.' in foaf_firstName:
                foaf_firstName = ''
        
    if foaf_firstName != '':
        name_without_prefix = foaf_firstName + ' ' + foaf_lastName
        
    if foaf_firstName in ['van', 'De']:
        
        foaf_lastName = foaf_firstName + ' ' + foaf_lastName
        foaf_firstName = ''
        
    if foaf_firstName.isupper() == True:
        foaf_initials = '.'.join(foaf_firstName)
        foaf_firstName = ''
       
    return foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes

In [102]:
def json_create_single_layer(organisatie_type):
    
    list_of_dicts = []
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Waterschap': 'ws',
                  'Gemeente': 'gm',
                   'Agentschap': 'oorg',
            
                  }
    afkorting = afkortingen[organisatie_type]
    
    tree = ET.parse("exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]


    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall(f'.//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)
    
    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:

        # In de resourceIdentifierTOOI staat de organisatiecode en veramel deze code
        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        
        org_json = create_organisatie_json(organisatie, namespaces, afkorting)
        
        # in het functie_element staan alle verschillende functies
        functie_element = organisatie.findall('p:functies/p:functie', namespaces)

        # loop over deze functies
        for functie in functie_element:

            foi_count = 0
            DICT = {}

            foaf_function_type = xpath(functie, 'p:naam', namespaces)

            # vindt alle medewerkers die bij deze functie horen
            medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

            # loop over alle medewerkers om per medewerker gegevens te verzamelen
            for medewerker in medewerker_element:
                
                foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)
                
                # optioneel startdatum, telefoonnummer en mailadres
                foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)
                
                foi_bezoekadres = xpath_adres(medewerker, namespaces, 'Bezoek')
                
                foi_postadres = xpath_adres(medewerker, namespaces, 'Post')
                
                # genereer beschikbaarheidsgegevens
                if organisatie_type == 'Waterschap':
                    bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {Type.lower()} {dc_publisher_name}"
                else:
                    if foi_party != '':
                        bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {foi_party} in de {dc_publisher_name}"
                    else:
                        bereikbaarheidsgegevens = f"bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {dc_publisher_name}"
                
#                 query = f"{name_without_prefix} {foaf_function_type} '{dc_publisher_name}'"
                
#                 hrefs = search_yahoo(query)
    
#                 foi_twitter, foi_website, foi_linkedin = extract_urls(hrefs, website_txt, foaf_lastName)
        
#                 if foi_linkedin != '':
#                     if '/in/' not in foi_linkedin:
#                         foi_linkedin = ''
                    
#                     if '/' in foi_linkedin:
#                         if foi_linkedin.split('/')[-1][0] != foaf_initials[0].lower():
#                             foi_linkedin = ''
                
#                 if foi_website != '':
#                     last_url = foi_website.split('/')
#                     last_url = [item for item in last_url if item]
        
#                     words_on_last_url = last_url[-1].split('-')
                    
#                     score = 0
#                     items = [foaf_lastName, foaf_function_type, dc_publisher_name]
#                     new_list = [split_item.lower() for item in items for split_item in re.split(r'\s|-', item) if split_item]        
#                     new_list = [word.strip(string.punctuation) for word in new_list]
                    
#                     for word in words_on_last_url:
#                         if word not in new_list:
#                             score = score + 1

#                     if score > 2:
#                         foi_website = ''
                                
                    
                # vul de dictionary in
                Dict = {
                        'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                        'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                        'dc_type': Type,
                        'dc_description': bereikbaarheidsgegevens,
                        'dc_source': f"https://organisaties.overheid.nl/archive/exportOO_{organisatie_type.lower()}.xml",
                        'dc_publisher': dc_publisher,
                        'dc_creator': "R0m4ndu",
                        'foi_retrievedDate': foi_retrievedDate,
                        'dc_date_year': dc_date_year,
#                         'foi_worksFor': f"nl.{dc_publisher}"
                        'dc_publisher_name': dc_publisher_name,
                        'foi_title': prefixes,
                        'foaf_initials': foaf_initials,
                        'foaf_firstName': foaf_firstName,
                        'foaf_lastName': foaf_lastName,
                        'foaf_name': name_without_prefix,
                        'foaf_mbox': foaf_mbox,
                        'foaf_phone': foaf_phone,
                        'foi_visitAddress': foi_bezoekadres,
                        'foi_mailAddress': foi_postadres,
#                         'foi_linkedin': foi_linkedin,
#                         'foaf_workplaceHomepage': foi_website,
                        'foi_startDate': foi_startDate,
                        'foi_party': foi_party,
                        'foi_function': foaf_function_type,
                        'foi_files': [] ,
                    }
                # Verwijder alle lege strings uit de dict
                filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                # en voeg de dictionary toe aan 
                DICT[foi_count] = filtered_dict

                foi_count+=1
            
            if dc_publisher != '':
            
                # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
                final_dict = {'resource': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}",
                            'infobox': {'foi_totalDossiers': len(DICT),
                                         'foi_dossiers': DICT}}


                list_of_dicts.append(final_dict)
                
            

    number_of_people = sum(i['infobox']['foi_totalDossiers'] for i in list_of_dicts)
    
    Json = json.dumps(list_of_dicts, indent=4)

    # Write JSON string to a text file
    with open(f"{organisatie_type}.json", "w") as file:
        file.write(Json)
        
    return number_of_people, list_of_dicts


In [103]:
def json_create_multi_layer(organisatie_type):
    
    list_of_dicts = []
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Provincie': 'pv',
                  'Ministerie': 'mnre',
                  'Hoog College van Staat': 'oorg',
                  'Rechtspraak': ''}
    afkorting = afkortingen[organisatie_type]
    
    tree = ET.parse(f"exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]

    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall(f'.//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)

    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:

        # In de resourceIdentifierTOOI staat de organisatiecode en veramel deze code
        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url''p:types/p:type', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        
        organisatie_elements2 = organisatie.findall('.//p:organisaties/p:organisatie', namespaces)
        
        for organisatie2 in organisatie_elements2:
            
            organisatie_naam = xpath(organisatie2, 'p:naam', namespaces)
        
            # in het functie_element staan alle verschillende functies
            functie_element = organisatie2.findall('p:functies/p:functie', namespaces)


            # loop over deze functies
            for functie in functie_element:
                foi_count = 0
                DICT = {}

                foaf_function_type = xpath(functie, 'p:naam', namespaces)

                # vindt alle medewerkers die bij deze functie horen
                medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

                # loop over alle medewerkers om per medewerker gegevens te verzamelen
                for medewerker in medewerker_element:

                    foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                    foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)


                    # optioneel startdatum, telefoonnummer en mailadres
                    foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                    foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                    foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)

                    # genereer beschikbaarheidsgegevens

                    if organisatie_type != 'Provincies':
                        if foi_party != '':
                            bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {foi_party} in de {dc_publisher_name}"
                        else:
                            bereikbaarheidsgegevens = f"bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {dc_publisher_name}"

                    # vul de dictionary in
                    Dict = {
                            'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                            'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                            'dc_type': Type,
                            'dc_description': bereikbaarheidsgegevens,
                            'dc_source': f"https://organisaties.overheid.nl/archive/exportOO_{organisatie_type.lower()}.xml",
                            'dc_publisher': dc_publisher,
                            'dc_creator': "R0m4ndu",
                            'foi_retrievedDate': foi_retrievedDate,
                            'dc_date_year': dc_date_year,
                            'dc_publisher_name': dc_publisher_name,
#                             'foi_worksFor': f"nl.{dc_publisher}",
                            'foi_title': prefixes,
                            'foaf_initials': foaf_initials,
                            'foaf_firstName': foaf_firstName,
                            'foaf_lastName': foaf_lastName,
                            'foaf_name': name_without_prefix,
                            'foaf_mbox': foaf_mbox,
                            'foaf_phone': foaf_phone,
                            'foi_startDate': foi_startDate,
                            'foi_party': organisatie_naam,
                            'foi_function': foaf_function_type,
                            'foi_files': [] ,
                        }

                    # Verwijder alle lege strings uit de dict
                    filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                    # en voeg de dictionary toe aan 
                    DICT[foi_count] = filtered_dict

                    foi_count+=1
                
                if dc_publisher != '':
                
                    # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
                    final_dict = {'resource': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}",
                                'infobox': {'foi_totalDossiers': len(DICT),
                                             'foi_dossiers': DICT}}


                    list_of_dicts.append(final_dict)

    number_of_people = sum(i['infobox']['foi_totalDossiers'] for i in list_of_dicts)
    
    Json = json.dumps(list_of_dicts, indent=4)

    # Write JSON string to a text file
    with open(f"{organisatie_type}_json.txt", "w") as file:
        file.write(Json)
        
    return number_of_people, list_of_dicts

In [104]:
def create_organisatie_json(organisatie, namespaces, afkorting):
    
    foi_bezoekadres = xpath_adres(organisatie, namespaces, 'Bezoek')
    foi_postadres = xpath_adres(organisatie, namespaces, 'Post')
    
    foaf_phone = xpath(organisatie, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

    foaf_mbox = xpath(organisatie, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)
    
    foi_fax = xpath(organisatie, 'p:contact/p:fax', namespaces)
    
    foi_lastUpdate = xpath(organisatie, 'p:datumMutatie', namespaces)
    
    dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
    dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

    website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url', namespaces)

    Type = xpath(organisatie, 'p:types/p:type', namespaces)
    
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    dc_date_year = foi_retrievedDate[:4]
    
    foi_startDate = xpath(organisatie, 'p:startDatum', namespaces)
    
    foi_endDate = xpath(organisatie, 'p:eindDatum', namespaces)
    
    Dict = {
        'dc_identifier': f"nl.{dc_publisher}",
        'dc_title': f"{Type} - {dc_publisher_name}",
        'dc_publisher': dc_publisher,
        'dc_creator': "Ramon Duursma",
        'foi_retrievedDate': foi_retrievedDate,
        'dc_date_year': dc_date_year,
        'dc_publisher_name': dc_publisher_name,
        'foi_website': website_txt,
        'foaf_mbox': foaf_mbox,
        'foaf_phone': foaf_phone,
        'foi_bezoekadres': foi_bezoekadres,
        'foi_postadres': foi_postadres,
        'foi_fax': foi_fax,
        'foi_startDate': foi_startDate,
        'foi_endDate': foi_endDate,
        'foi_lastUpdate': foi_lastUpdate,
        'foaf_type': Type,
        'foi_files': [],
    }
    
    
    filtered_dict = {key: value for key, value in Dict.items() if value != ""}
    return filtered_dict
    

In [105]:
def json_alle_organisaties(Type = ''):
    org_dict = {}
    
    Types = []
    TOOIs = []
    
    count = 0
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Waterschap': 'ws',
                  'Gemeente': 'gm',
                   'Provincie': 'pv',
                   'Ministerie': 'mnre',
                  }
    
    tree = ET.parse("exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }
    
    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    if Type == '':
        organisatie_elements = tree.findall(f'/p:organisaties/p:organisatie', namespaces)
    
    else:
        organisatie_elements = tree.findall(f'./p:organisaties/p:organisatie/[p:types = "{Type}"]', namespaces)
    
    for organisatie  in organisatie_elements:
        
        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        
        TOOI = organisatie.find('.//p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces)
        Code = organisatie.find('.//p:identificatiecodes/p:resourceIdentifier[@p:naam="Organisatiecode"]', namespaces)
            
        if Type in afkortingen:
            afkorting = afkortingen[Type]
            
        else:
            afkorting = ''            
            
        Dict = create_organisatie_json(organisatie, namespaces, afkorting)
        
        if 'foi_endDate' not in Dict:
            org_dict[count] = Dict
            Types.append(Type)
            count+=1
        else:
            pass
        
    final_dict = {'infobox':
                    {
                        'foi_totalDossiers': count,
                        'foi_dossiers': org_dict,
                    }
                 }
        
    
    Json = json.dumps(final_dict, indent=4)

    # Write JSON string to a text file
    with open(f"alle_organisaties_json.txt", "w") as file:
        file.write(Json)
    
    return final_dict, Types, TOOIs
    
final_dict, Types, TOOIs = json_alle_organisaties()

C:\Users\ramon\AppData\Local\Temp\ipykernel_19364\2391481212.py:25: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './p:organisaties/p:organisatie'
  organisatie_elements = tree.findall(f'/p:organisaties/p:organisatie', namespaces)


In [106]:
def json_medewerkers(Type = ''):
    
    DICT = {}
    
    foi_count = 0
    
    tree = ET.parse("exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }
    
    afkortingen = {'Waterschap': 'ws',
                  'Gemeente': 'gm',
                   'Provincie': 'pv',
                   'Ministerie': 'mnre'
                  }
    

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]

    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    
    if Type == '':
        organisatie_elements = tree.findall(f'.//p:organisaties/p:organisatie', namespaces)
    else:     
        organisatie_elements = tree.findall(f'./p:organisaties/p:organisatie/[p:types = "{Type}"]', namespaces)
    
    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        
        if Type in afkortingen:
            afkorting = afkortingen[Type]
        else:
            afkorting = ''

        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 

        
        # in het functie_element staan alle verschillende functies
        functie_element = organisatie.findall('p:functies/p:functie', namespaces)

        # loop over deze functies
        for functie in functie_element:

            foaf_function_type = xpath(functie, 'p:naam', namespaces)

            # vindt alle medewerkers die bij deze functie horen
            medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

            # loop over alle medewerkers om per medewerker gegevens te verzamelen
            for medewerker in medewerker_element:
                
                foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)
                
                # optioneel startdatum, telefoonnummer en mailadres
                foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)
                
                foi_bezoekadres = xpath_adres(medewerker, namespaces, 'Bezoek')
                
                foi_postadres = xpath_adres(medewerker, namespaces, 'Post')
            
                # vul de dictionary in
                Dict = {
                        'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                        'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                        'foi_function': foaf_function_type,
                        'dc_source': f"https://organisaties.overheid.nl/archive/exportOO_{Type.lower()}.xml",
                        'dc_creator': "Ramon Duursma",
                        'foi_retrievedDate': foi_retrievedDate,
                        'dc_date_year': dc_date_year,
                        'dc_publisher': dc_publisher,
                        'dc_publisher_name': dc_publisher_name,
                        'foaf_initials': foaf_initials,
                        'foaf_firstName': foaf_firstName,
                        'foaf_lastName': foaf_lastName,
                        'foi_title': prefixes,
                        'foaf_name': name_without_prefix,
                        'foaf_mbox': foaf_mbox,
                        'foaf_phone': foaf_phone,
                        'foi_bezoekadres': foi_bezoekadres,
                        'foi_postadres': foi_postadres,
                        'foaf_type': Type,
                        'foi_startDate': foi_startDate,
                        'foi_party': foi_party,
                        'foi_files': [] ,
                    }

                # Verwijder alle lege strings uit de dict
                filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                # en voeg de dictionary toe aan 
                DICT[foi_count] = filtered_dict

                foi_count+=1
            
    # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
    final_dict = {
                'infobox': {'foi_totalDossiers': len(DICT),
                             'foi_dossiers': DICT}}
                
            

    number_of_people = final_dict['infobox']['foi_totalDossiers']
    
    Json = json.dumps(final_dict, indent=4)

    # Write JSON string to a text file
    with open(f"alle_medewerkers_json.txt", "w") as file:
        file.write(Json)
        
    return number_of_people, final_dict


In [107]:
thing_series = pd.Series(Types)

# Use value_counts() to get the count of each thing
counts = thing_series.value_counts().reset_index()

# Rename the columns
counts.columns = ['Type Organisatie', 'aantal']

all_types = counts['Type Organisatie'].to_list()
all_counts = []

tree = ET.parse("exportOO_full.xml")

# Define the namespaces used in the XML document
namespaces = {
'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
}

for Type in all_types:
    
    mc = 0


    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    org_elements = tree.findall(f'./p:organisaties/p:organisatie/[p:types = "{Type}"]', namespaces)
    
    for org in org_elements:
        medewerkers = org.findall('.//p:medewerkers/p:medewerker', namespaces)
        
        mc += len(medewerkers)
    
    
    all_counts.append(mc)
    
medewerker_counts = dict(zip(all_types, all_counts))

# Vanaf hier beginnen de tabellen!
# ----------------------------------------------

In [108]:
counts['medewerkers'] = medewerker_counts.values()

new_row = pd.DataFrame({'Type Organisatie': ['Totaal:'], 'aantal': [sum(counts['aantal'])], 'medewerkers': [sum(counts['medewerkers'])]})

counts = counts.append(new_row, ignore_index=False)

counts.index = counts.index+1

counts['m/o'] = counts['medewerkers'] / counts['aantal']

counts['m/o'] = counts['m/o'].round(1)

display(counts)

,Type Organisatie,aantal,medewerkers,m/o
1,Organisatie met overheidsbemoeienis,1367,0,0.0
2,Regionaal samenwerkingsorgaan,348,153,0.4
3,Gemeente,342,14120,41.3
4,Zelfstandig bestuursorgaan,137,0,0.0
5,Adviescollege,93,30,0.3
6,Agentschap,29,68,2.3
7,Rechtspraak,25,69,2.8
8,Waterschap,21,42,2.0
9,Provincie,12,339,28.2
10,Ministerie,12,741,61.8


Table 1: Inmiddels zijn de spookorganisaties eruit verwijderd. Eigenlijk 4 waterschappen en een deel van de oude gemeenten. Er zitten nu dus nog wat oude gemeenten in, maar om deze te vinden moet je eigenlijk kijken naar gemeenten zonder medewerkers, maar in zekere zin zou je dan organisaties zonder medewerkers eruit moeten filteren maar dan filter je er ook 5 provincies uit en waarschijnlijk 80% van alle andere organisaties. Maar opzich kan het aantal gemeenten nog wel handmatig gemanipuleerd worden in Latex. (EDIT foi_endDate bepaald natuurlijk of een organisatie wel of niet meer bestaat dus alles met een end_date kan verwijderd worden!)

Opvallende zaken: In eerste instantie geeft dit een goed beeld aan de lezer voor een overzicht van de hoeveelheden van organisaties waar we mee te maken krijgen en in de forse verschillen in frequentie van organisaties tussen organisatietypen. Hetzelfde geld eigenlijk voor de medewerkers waar veruit het grootste deel aan de gemeenten toebehoort. Het valt ook op dat er een aantal organisatietypen zijn met geen of weinig medewerkers. Hierdoor valt het gebrui van de medewerkers voor een deel van deze organisaties grotendeels af. En het geeft aan welke organisaties wel interessant zijn voor verdere analyse.

In [109]:
identifiers = ['dc_publisher', 'foaf_mbox', 'foaf_phone', 'foi_website', 'foi_fax', 'foi_startDate', 'foi_bezoekadres', 
               'foi_postadres', 'foi_lastUpdate', 'foaf_type']

id_dict = {key: 0 for key in identifiers}

FD, Types, TOOIs = json_alle_organisaties()

dossiers = FD['infobox']['foi_dossiers']
for k,v in dossiers.items():
    for ID in identifiers:
        if ID in v:
            id_dict[ID] += 1
            
df = pd.DataFrame.from_dict(id_dict, orient='index', columns=['Count'])
            
for Type in ['Organisatie met overheidsbemoeienis', 'Regionaal samenwerkingsorgaan','Gemeente']:

    FD, Types, TOOIs = json_alle_organisaties(Type = Type)

    id_dict = {key: 0 for key in identifiers}

    dossiers = FD['infobox']['foi_dossiers']
    for k,v in dossiers.items():
        for ID in identifiers:
            if ID in v:
                id_dict[ID] += 1

    df2 = pd.DataFrame.from_dict(id_dict, orient='index', columns=['Count'])

    df[Type] = df2['Count']
    
    
display(df)

C:\Users\ramon\AppData\Local\Temp\ipykernel_19364\2391481212.py:25: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './p:organisaties/p:organisatie'
  organisatie_elements = tree.findall(f'/p:organisaties/p:organisatie', namespaces)


,Count,Organisatie met overheidsbemoeienis,Regionaal samenwerkingsorgaan,Gemeente
dc_publisher,606,0,0,342
foaf_mbox,801,6,303,333
foaf_phone,908,11,343,335
foi_website,1106,175,325,342
foi_fax,71,0,31,0
foi_startDate,739,4,348,122
foi_bezoekadres,902,17,343,342
foi_postadres,1426,597,296,341
foi_lastUpdate,2428,1367,348,342
foaf_type,2428,1367,348,342


In tabel 2 is te zien hoevaak een bepaalde metadate bekend is voor organisaties. De eerste kolom is het totaal voor alle organisaties, maar om het verschil tussen bepaalde organisaties te laten zien is ook gekozen om de drie organisatietypen met de meeste organisaties ook mee te nemen. Het valt hier op dat voor de organisaties met overheidsbemoeienis er eigenlijk bijna nooit echt veel contact gegevens bekend zijn (behalve postadres en soms website). Terwijl dit voor de regionale samenwerkingsorganen en de gemeentes een stuk vaker wel bekend is. Het laat dus zien dat voor elke organisatie weer andere soorten gegevens beschikbaar zijn. En dat het totaal van alle organisaties grotendeels gedrukt wordt door de OMO's. 

In [110]:
identifiers = ['foi_function', 'dc_publisher', 'foaf_initials', 'foaf_firstName', 'foaf_lastName', 'foi_title',
'foaf_mbox', 'foaf_phone', 'foi_bezoekadres', 'foi_postadres' , 'foaf_type', 'foi_startDate', 'foi_party']

id_dict = {key: 0 for key in identifiers}

NP, fd = json_medewerkers()

dossiers = fd['infobox']['foi_dossiers']
for k,v in dossiers.items():
    for ID in identifiers:
        if ID in v:
            id_dict[ID] += 1
            
df = pd.DataFrame.from_dict(id_dict, orient='index', columns=['Totaal'])

NP, fd2 = json_medewerkers(Type = 'Gemeente')

id_dict = {key: 0 for key in identifiers}

dossiers = fd2['infobox']['foi_dossiers']
for k,v in dossiers.items():
    for ID in identifiers:
        if ID in v:
            id_dict[ID] += 1
            
df2 = pd.DataFrame.from_dict(id_dict, orient='index', columns=['Gem'])
            
df['Gemeente'] = df2['Gem']
df['Zonder Gemeente'] = df['Totaal'] - df['Gemeente']

display(df)

,Totaal,Gemeente,Zonder Gemeente
foi_function,16044,14120,1924
dc_publisher,14290,14120,170
foaf_initials,15983,14119,1864
foaf_firstName,579,0,579
foaf_lastName,16044,14120,1924
foi_title,14992,13998,994
foaf_mbox,14058,13602,456
foaf_phone,14168,13758,410
foi_bezoekadres,14325,14120,205
foi_postadres,14489,14087,402


Vergelijkbaar met de vorige tabel, maar hier met de metadata dat bekend is per persoon. En omdat medewerkers van gemeenten veruit het grootste deel bevatten zijn deze ook apart in de tabel te vinden. (+ alle overige). Het viel hier op dat de gegevens vrij compleet waren op 2 gegevens na. Dit komt natuurlijk door het feit dat gemeenten de meeste gegevens vrijwel altijd verzamelen, maar nooit de voornaam of de startdatum.V Voor alle organisaties samen is te zien dat er vaak gegevens missen, voor bijna alle type gegevens. 

In [111]:
def create_functie_table(organisatie):
    if organisatie in ['Provincie', 'Ministerie']:
        people_num, dict_list = json_create_multi_layer(organisatie)
        
    else:    
        people_num, dict_list = json_create_single_layer(organisatie)

    data = []

    for d in dict_list:

        doi_split = d['resource'].split('.')
        publisher = d['infobox']['foi_dossiers'][0]['dc_publisher_name']
        functie = doi_split[2]
        personen = d['infobox']['foi_totalDossiers']


        functie_omzetten = {
            'Dijkgraaf': 'Dijk/Watergraaf',
            'Watergraaf': 'Dijk/Watergraaf',
            'Secretaris - Algemeen directeur': 'Secretaris-directeur',
            'Voorzitter': 'Dijk/Watergraaf',
            'Burgemeester waarnemend': 'Burgemeester',
            'Adjunct-hoofd': 'Adjunct hoofd',
            'Ádjunct-hoofdeenheid': 'Adjunct hoofd',
            'Commissaris van de koning': 'Commissaris van de Koning',
            'CdK': 'Commissaris van de Koning',
            'CvdK': 'Commissaris van de Koning'
            
        }

        if functie in functie_omzetten.keys():
            functie = functie_omzetten[functie]

        data.append({'Publisher': publisher, 'Functie': functie, 'Count': personen})

    # Create a DataFrame from the data
    df = pd.DataFrame(data)

    # Pivot the table
    table = pd.pivot_table(df, values='Count', columns='Functie', index='Publisher', aggfunc='sum', fill_value=0)

    return table


In [112]:
waterschap_table = create_functie_table('Waterschap')
wt_desc = waterschap_table.describe()
rows_to_drop = ['25%', '50%', '75%']
wt_desc = wt_desc.drop(rows_to_drop)

display(wt_desc)

Functie,Dijk/Watergraaf,Secretaris-directeur
count,21.0,21.0
mean,1.0,1.0
std,0.0,0.0
min,1.0,1.0
max,1.0,1.0


Dit zijn de descriptives van de "hoeveel mensen hebben functie X van alle organisaties binnen het type. In dit geval dus van de waterschappen. Uit deze gegevens blijkt dat de functies Dijk/Watergraaf en Secretaris-Directeur de enige twee functies bij waterschappen zijn die bekend in het XML en bij ieder waterschap is er 1 van elk. Precies wat we willen dus!"

In [113]:
gemeente_table = create_functie_table('Gemeente')

pd.reset_option('display.max_rows')
gm_desc = gemeente_table.describe()
rows_to_drop = ['25%', '50%', '75%']
gm_desc = gm_desc.drop(rows_to_drop)

display(gm_desc)

Functie,Burgemeester,Fractievoorzitter,Gemeentesecretaris,Locoburgemeester,Raadsgriffier,Raadslid,Wethouder
count,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000
mean,1.002907,8.154070,0.976744,0.956395,1.002907,24.834302,4.119186
std,0.053916,2.617966,0.150934,0.204511,0.053916,8.081862,1.195367
min,1.000000,3.000000,0.000000,0.000000,1.000000,9.000000,1.000000
max,2.000000,16.000000,1.000000,1.000000,2.000000,45.000000,9.000000


Hier is een vergelijkbare descriptives table maar dan voor de gemeenten. Hier valt op dat er net iets meer dan 1 burgemeester en raadsgriffier zijn per gemeente. Het blijkt uiteindelijk dat dit er twee zijn in 1 gemeente voor elk. Bij de locoburgemeester en gemeentesecretaris valt op dat er in een aantal gemeenten 0 zijn. Wellicht missende data of er bestaat gewoon niemand met die functie. De gegevens over de raadsleden zijn zeer plausibel te noemen. Het minimum aantal raadsleden voor een gemeente met minder dan 3000 inwoners (Schiermonnikoog) is 9 en maximaal 45 voor een gemeente met meer dan 200.000 inwoners hetzelfde min en max blijken uit de tabel. Verder is er een recente bron die aangeeft dat er 4.14 wethouders gemiddeld per gemeente zijn. Het aantal wat hier is aangetroffen is slechts iets lager dus het kan zijn dat er een hand vol missen of dat er wat wethouders weg zijn gegaan, maar prima resultaten al met al. 

In [114]:
provincie_table = create_functie_table('Provincie')

pd.set_option('display.max_columns', None)
provincie_table.describe()
pd.reset_option('display.max_columns')

table_cols = provincie_table.columns.tolist()
print(len(table_cols))

p_sums = provincie_table.sum(axis=1)
display(p_sums)

38


Publisher
Provincie Drenthe           3
Provincie Flevoland         3
Provincie Fryslân           6
Provincie Groningen        92
Provincie Noord-Holland    65
Provincie Overijssel       77
Provincie Zuid-Holland     93
dtype: int64

Het aantal verschillende functies (38) geeft aan dat het niet bepaald mogelijk is om een overzichtelijke desciptives tabel te geven hierom heb ik gekozen voor het aantal functies per provincie. Dit laat zien dat er sowieso al 5 provincies ontbreken en dat er nog eens 3 provincies zijn met een verdacht laag aantal medewerkers (raadsleden zullen er sowieso niet instaan). Daarnaast worden de overgebleven 4 organisaties geanalyseerd en wordt beargumenteerd dat alleen Noord-Holland een correcte hoeveelheid raadsleden kan hebben (en dus later gebruikt kan worden voor steekproef). 

In [115]:
provincie_table = create_functie_table('Ministerie')

pd.set_option('display.max_columns', None)
provincie_table.describe()
pd.reset_option('display.max_columns')

table_cols = provincie_table.columns.tolist()
print(len(table_cols))

p_sums = provincie_table.sum(axis=1)
display(p_sums)

267


Publisher
Binnenlandse Zaken en Koninkrijksrelaties     55
Defensie                                     103
Economische Zaken en Klimaat                   4
Financiën                                      2
Justitie en Veiligheid                       255
Landbouw, Natuur en Voedselkwaliteit           4
Onderwijs, Cultuur en Wetenschap             165
Sociale Zaken en Werkgelegenheid               1
Volksgezondheid, Welzijn en Sport             99
dtype: int64

Deze tabel lijkt heel erg op de provincie tabel. Het blijkt dat er wederom ministeries missen en dat er ministeries zijn met een lachwekkend laag aantal medewerkers. Omdat het aantal verschillende functies met (267) zo hoog is is het bijna onmogelijk om hier iets van overzicht in te krijgen en al helemaal om een steekproef te maken of alle functies kloppen etc. 

In [116]:
data = {
    'Organisatie': ['Terschelling', 'Hoorn', 'Amsterdam', 'Noord-Holland', 'Aa en Maas', 'Rivierenland'],
    'Medewerkers in XML': [23, 57, 71, 65, 2, 2],
    'Medewerkers Actueel': [22, 56, 71, 63, 2, 2]
}

df = pd.DataFrame(data)
display(df)

,Organisatie,Medewerkers in XML,Medewerkers Actueel
0,Terschelling,23,22
1,Hoorn,57,56
2,Amsterdam,71,71
3,Noord-Holland,65,63
4,Aa en Maas,2,2
5,Rivierenland,2,2


Deze tabel is logischerwijs handmatig ingevuld dus ik heb hem er gewoon met pandas ingezet. In deze tabel is een steekproef genomen van een aantal organisaties te vinden in de XML. Ik heb gekozen voor een kleine gemeente, een middelgrote en een grote. En voor de enige bruikbare provincie. Hieruit blijkt dat de gegevens eigenlijk erg actueel zijn. Bij gemeenten was er twee keer onduidelijkheid over de locoburgemeester het lijkt erop als ze de tweede locoburgemeester in lijn hebben genoteerd i.p.v. de eerste. Dus wellicht een aanbeveling dat gemeenten hier nog naar kijken voor de XML maar verder niks op aan te merken. Noord-Holland was ook redelijk accuraat, maar het bleek wel dat onlangs twee statenleden zijn vervangen dus het is ietswat gedateerd maar geen drama. Waterschappen zijn maar 2 personen per org. maar daar was niks op aan te merken. 